## New results Binary

### train 1000, eval 1000

```
================== (EPOCH 1)
accuracy_score  : 0.947
f1_score  : 0.9494756911344138
balanced_accuracy_score  : 0.9464011147414952
average_precision_score  : 0.9197409136524282
balanced_accuracy_score  : 0.9464011147414952
================== (EPOCH 5) SPLIT on 0.5
balanced_accuracy_score  : 0.958861874559549
average_precision_score  : 0.9413499275589611
balanced_accuracy_score  : 0.958861874559549
accuracy_score  : 0.959
f1_score  : 0.9603864734299518
[[462  22]
 [ 19 497]]
Acc 0.959 Prec 0.9576107899807321 Rec 0.9631782945736435 F1 0.9603864734299518
================== (EPOCH 15) SPLIT on 0.6
balanced_accuracy_score  : 0.9577647511051317
average_precision_score  : 0.9387431168136862
balanced_accuracy_score  : 0.9577647511051317
accuracy_score  : 0.958
f1_score  : 0.9595375722543352
[[460  24]
 [ 18 498]]
Acc 0.958 Prec 0.9540229885057471 Rec 0.9651162790697675 F1 0.9595375722543352
============SPLIT on  0.5 (after using pre-processing)
balanced_accuracy_score  : 0.943486129796912
average_precision_score  : 0.9280883783070015
balanced_accuracy_score  : 0.943486129796912
accuracy_score  : 0.943
f1_score  : 0.9438423645320196
[[464  20]
 [ 37 479]]
Acc 0.943 Prec 0.9599198396793587 Rec 0.9282945736434108 F1 0.9438423645320196
```
### train 20000, eval 1000, epoch=2
```
============SPLIT on  0.6
accuracy_score  : 0.978
f1_score  : 0.978805394990366
balanced_accuracy_score  : 0.9777852520981485
average_precision_score  : 0.9660920134248121
balanced_accuracy_score  : 0.9777852520981485
```
### train 20000, eval 1000, epoch=2
```
============SPLIT on  0.2
accuracy_score  : 0.982
f1_score  : 0.9825581395348837
balanced_accuracy_score  : 0.9819815491062849
average_precision_score  : 0.9744204975662519
balanced_accuracy_score  : 0.9819815491062849
============SPLIT on  0.5
accuracy_score  : 0.979
f1_score  : 0.9795918367346939
balanced_accuracy_score  : 0.9790745723620988
average_precision_score  : 0.9716083231334148
balanced_accuracy_score  : 0.9790745723620988
```
### train 60000, eval 1000, epoch=2
```
============SPLIT on  0.4
balanced_accuracy_score  : 0.9779133833045037
average_precision_score  : 0.9679030858100626
balanced_accuracy_score  : 0.9779133833045037
accuracy_score  : 0.978
f1_score  : 0.9787234042553191
[[472  12]
 [ 10 506]]
Acc 0.978 Prec 0.9768339768339769 Rec 0.9806201550387597 F1 0.9787234042553191
```

## old results

### learning on 1000, eval on 10000
```
drop=0.1
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 100) (1000, 1)
[ 0  1  2  3  4  6  7  8 10 12] [ 5  9 11 16 37 38 39 43 62 63]
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2423
mean_absolute_error on #1 : 0.0675381
mean_squared_error on #1 : 0.037668515
r2 score on #1 : 0.849319242356755
new acc >>  0.037668515
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 4e-05
(6, 1000, 100) (1000, 1)
[ 0  1  2  3  4  6  7  8 10 12] [ 5  9 11 16 37 38 39 43 62 63]
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.2018
mean_absolute_error on #1 : 0.073092975
mean_squared_error on #1 : 0.039947953
r2 score on #1 : 0.840201083547445
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 7.000000000000001e-05
(6, 1000, 100) (1000, 1)
[ 0  1  2  3  4  6  7  8 10 12] [ 5  9 11 16 37 38 39 43 62 63]
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.2241
mean_absolute_error on #1 : 0.131146
mean_squared_error on #1 : 0.07735249
r2 score on #1 : 0.6905762882973068



drop=0.18
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.2294
mean_absolute_error on #1 : 0.080276355
mean_squared_error on #1 : 0.049829144
r2 score on #1 : 0.8006745685035439
new acc >>  0.049829144
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 4e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2043
mean_absolute_error on #1 : 0.052972056
mean_squared_error on #1 : 0.030377457
r2 score on #1 : 0.8784847705266939
new acc >>  0.030377457
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 7.000000000000001e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.1986
mean_absolute_error on #1 : 0.103421785
mean_squared_error on #1 : 0.04079107
r2 score on #1 : 0.8368284719652488



drop=0.2
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 39s 39ms/sample - loss: 0.2252
mean_absolute_error on #1 : 0.07237325
mean_squared_error on #1 : 0.042013403
r2 score on #1 : 0.8319389297231697
new acc >>  0.042013403
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 4e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.1975
mean_absolute_error on #1 : 0.05105091
mean_squared_error on #1 : 0.03530891
r2 score on #1 : 0.8587580810612465
new acc >>  0.03530891
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 7.000000000000001e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2140
mean_absolute_error on #1 : 0.059229802
mean_squared_error on #1 : 0.034580465
r2 score on #1 : 0.8616719906285091
new acc >>  0.034580465


drop=0.22
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 39s 39ms/sample - loss: 0.2457
mean_absolute_error on #1 : 0.08117449
mean_squared_error on #1 : 0.044529174
r2 score on #1 : 0.8218753896231156
new acc >>  0.044529174
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 4e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2123
mean_absolute_error on #1 : 0.055401854
mean_squared_error on #1 : 0.034017615
r2 score on #1 : 0.8639234792592749
new acc >>  0.034017615
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 7.000000000000001e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.2174
mean_absolute_error on #1 : 0.08069304
mean_squared_error on #1 : 0.055955153
r2 score on #1 : 0.7761694301790417



DROP=0.3
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 37s 37ms/sample - loss: 0.3073
mean_absolute_error on #1 : 0.09093676
mean_squared_error on #1 : 0.046078745
r2 score on #1 : 0.8156768301002534
new acc >>  0.046078745
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 4e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2203
mean_absolute_error on #1 : 0.07295324
mean_squared_error on #1 : 0.03520016
r2 score on #1 : 0.8591931078777739
new acc >>  0.03520016
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 7.000000000000001e-05
(6, 1000, 100) (1000, 1)
Train on 1000 samples
1000/1000 [==============================] - 38s 38ms/sample - loss: 0.2278
mean_absolute_error on #1 : 0.056572042
mean_squared_error on #1 : 0.03633268
r2 score on #1 : 0.8546628036950575
```

In [1]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
cpu MHz		: 2000.134
cpu cores	: 1
# RAM
MemTotal:       16430180 kB
# GPU

# OS
Linux 51694d7a520d 4.19.112+ #1 SMP Sat Apr 4 00:12:45 PDT 2020 x86_64 GNU/Linux


In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

2.1.0


# Choose model

In [3]:
from transformers import BertTokenizer, BertModel


MODEL_TYPE = 'bert-base-uncased'
MAX_SIZE = 200
BATCH_SIZE = 200

tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [4]:
HAS_ANS = False
training_sample_count = 1000 # 4000
training_epochs = 1 # 3
test_count = 1000
running_folds = 1 # 2
MAX_SEQUENCE_LENGTH = 200

In [5]:
!ls /kaggle/input/200k-short-texts-for-humor-detection

dataset.csv  dev.csv  test.csv	train.csv


In [6]:
!ls /kaggle/input/cbert-after-preprocessing-by-1

best_birnn_attention_model.hdf5      best_cnn_model.hdf5  test_after_prep.csv
best_birnn_cnn_model.hdf5	     result_all.csv	  train_after_prep.csv
best_birnn_hierarchy_cnn_model.hdf5  submission.csv


### original dataset

In [7]:
df = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')

df_train = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/train.csv')
display(df_train.head(3))
df_train = df_train[:training_sample_count*5]

df_test = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dev.csv')
display(df_test.head(3))
df_test = df_test[:test_count]

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True


,text,humor
0,What kind of cat should you take into the des...,True
1,Remember when people used to have to be in sha...,True
2,Pizza is always good. - everyone we'll see abo...,True


### using pre-processing dataset

In [8]:
# df_train = pd.read_csv('/kaggle/input/cbert-after-preprocessing-by-1/train_after_prep.csv')
# display(df_train.head(3))
# df_train = df_train[:training_sample_count*5]

# df_test2 = pd.read_csv('/kaggle/input/cbert-after-preprocessing-by-1/test_after_prep.csv')
# df_test2['humor'] = df_test['humor']
# df_test = df_test2.copy()
# del df_test2
# display(df_test.head(3))
# df_test = df_test[:test_count]

### continue

In [9]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

200000 5000 1000


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


,text
0,What kind of cat should you take into the des...
1,Remember when people used to have to be in sha...
2,Pizza is always good. - everyone we'll see abo...
3,"What's 6 inches long hard, bent, and in my pan..."
4,Black teen's response to violence in his commu...


In [10]:
print(list(df_train.columns))

['text', 'humor']


In [11]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

if HAS_ANS:
    output_categories = list(df_train.columns[11:])
    input_categories = list(df_train.columns[[1,2,5]])
    

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['text']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


In [12]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title, None, 'longest_first', max_sequence_length)
    
    input_ids_a, input_masks_a, input_segments_a = return_id(
        '', None, 'longest_first', max_sequence_length)
        
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.text, instance.text, instance.text

        ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
        _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)
        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [13]:
outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)


## 3. Create model

~~`compute_spearmanr()`~~ `mean_squared_error` is used to compute the competition metric for the validation set
<br><br>
`create_model()` contains the actual architecture that will be used to finetune BERT to our dataset.


In [14]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
#     x = tf.keras.layers.Concatenate()([q, q])
    
    x = tf.keras.layers.Dropout(0.2)(q)
    
    x = tf.keras.layers.Dense(TARGET_COUNT, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, ], outputs=x)
    
    return model

#### 4. Obtain inputs and targets, as well as the indices of the train/validation splits

In [15]:
output_categories

['humor']

## 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 3 epochs --- with a learning rate of 3e-5 and batch_size of 6. A simple binary crossentropy is used as the objective-/loss-function. 

In [16]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
balanced_accuracy_score  : 0.5
average_precision_score  : 0.5
balanced_accuracy_score  : 0.5
accuracy_score  : 0.5
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

### Loss function selection
Regression problem between 0 and 1, so binary_crossentropy and mean_absolute_error seem good.

Here are the explanations: https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

In [17]:
min_acc = 1000000
min_test = []
valid_preds = []
test_preds = []
best_model = False
for LR in np.arange(1e-5, 2e-5, 3e-5).tolist():
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print('LR=', LR)
    gkf = GroupKFold(n_splits=5).split(X=df_train.text, groups=df_train.text)

    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold not in range(running_folds):
            continue
        train_inputs = [(inputs[i][train_idx])[:training_sample_count] for i in range(len(inputs))]
        train_outputs = (outputs[train_idx])[:training_sample_count]

        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = outputs[valid_idx]

        print(np.array(train_inputs).shape, np.array(train_outputs).shape)
#         print(train_idx[:10], valid_idx[:10])

        K.clear_session()
        model = create_model()
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
        model.compile(loss='binary_crossentropy', optimizer=optimizer)
        for xx in range(1):
            model.fit(train_inputs, train_outputs, epochs=training_epochs, batch_size=6)
            # model.save_weights(f'bert-{fold}.h5')
            valid_preds.append(model.predict(valid_inputs))
            #rho_val = compute_spearmanr_ignore_nan(valid_outputs, valid_preds[-1])
            #print('validation score = ', rho_val)
            acc = print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), 'on #'+str(xx+1))
            if acc < min_acc:
                print('new acc >> ', acc)
                min_acc = acc
                best_model = model
#                 min_test = model.predict(test_inputs)
#                 test_preds.append(min_test)
            print(' ')

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
LR= 1e-05
(6, 1000, 200) (1000, 1)



Train on 1000 samples
1000/1000 [==============================] - 48s 48ms/sample - loss: 0.2419
mean_absolute_error on #1 : 0.069623396
mean_squared_error on #1 : 0.03561097
r2 score on #1 : 0.8575196345624501
new acc >>  0.03561097
 


In [18]:
print('best acc >> ', acc)

best acc >>  0.03561097


In [19]:
len(valid_inputs[0])

1000

In [20]:
print(valid_outputs.shape, valid_preds[-1].shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), '')

(1000, 1) (1000, 1)
mean_absolute_error  : 0.069623396
mean_squared_error  : 0.03561097
r2 score  : 0.8575196345624501


0.03561097

In [21]:
# %%time
min_test = best_model.predict(test_inputs)

## use min_test
# min_test

In [22]:
df_test.to_csv('df_test.csv')
test_df_y.to_csv('test_df_y.csv')


## Regression submission

In [23]:
df_sub = test_df_y.copy()
# df_sub['pred'] = np.average(test_preds, axis=0) # for weighted average set weights=[...]
df_sub['pred'] = min_test


print_evaluation_metrics(df_sub['humor'], df_sub['pred'], '', True)


df_sub.to_csv('sub1.csv', index=False)
df_sub.head()

mean_absolute_error  : 0.074117444
mean_squared_error  : 0.039489683
r2 score  : 0.8418793658064543


,text,humor,pred
0,What kind of cat should you take into the des...,True,0.978363
1,Remember when people used to have to be in sha...,True,0.578363
2,Pizza is always good. - everyone we'll see abo...,True,0.975454
3,"What's 6 inches long hard, bent, and in my pan...",True,0.988652
4,Black teen's response to violence in his commu...,False,0.001187


## Binary submission

In [24]:
for split in np.arange(0.1, 0.99, 0.1).tolist():
    df_sub['pred_bi'] = (df_sub['pred'] > split)

    print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

    df_sub.to_csv('sub3.csv', index=False)
    df_sub.head()

================== SPLIT on 0.1
balanced_accuracy_score  : 0.9378163239156896
average_precision_score  : 0.9028866050450907
balanced_accuracy_score  : 0.9378163239156896
accuracy_score  : 0.939
f1_score  : 0.9428303655107779
[[436  48]
 [ 13 503]]
Acc 0.939 Prec 0.9128856624319419 Rec 0.9748062015503876 F1 0.9428303655107779
================== SPLIT on 0.2
balanced_accuracy_score  : 0.9422048177333591
average_precision_score  : 0.9122877133940258
balanced_accuracy_score  : 0.9422048177333591
accuracy_score  : 0.943
f1_score  : 0.9459715639810428
[[444  40]
 [ 17 499]]
Acc 0.943 Prec 0.9257884972170687 Rec 0.9670542635658915 F1 0.9459715639810428
================== SPLIT on 0.30000000000000004
balanced_accuracy_score  : 0.9486594272535076
average_precision_score  : 0.9254850498338871
balanced_accuracy_score  : 0.9486594272535076
accuracy_score  : 0.949
f1_score  : 0.9510086455331412
[[454  30]
 [ 21 495]]
Acc 0.949 Prec 0.9428571428571428 Rec 0.9593023255813954 F1 0.9510086455331412
===

# pun of the day

In [25]:
!ls /kaggle/input/reddithumordetection-all-datasets/full_datasets/puns

puns_pos_neg_data.csv


In [26]:
df_test = pd.read_csv('/kaggle/input/reddithumordetection-all-datasets/full_datasets/puns/puns_pos_neg_data.csv')
df_test.columns = ['humor', 'text']
df_test['humor'] = df_test['humor'] > 0
df_test

,humor,text
0,True,i used to be a banker but i lost interest
1,True,i m glad i know sign language it s pretty handy
2,True,i don t trust these stairs because they re alw...
3,True,did you hear about the guy who got hit in the ...
4,True,yesterday i accidentally swallowed some food c...
...,...,...
4821,False,Think much say little write less
4822,False,Though the fool waits the day does not
4823,False,Thrift is better than an annuity
4824,False,Thrive by honesty or remain poor


In [27]:
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [28]:
# %%time
min_test = best_model.predict(test_inputs)

df_test['pred'] = min_test

print_evaluation_metrics(df_test['humor'], df_test['pred'], '', True)

df_test.to_csv('puns.csv', index=False)
df_test.head()

mean_absolute_error  : 0.5176833
mean_squared_error  : 0.39698026
r2 score  : -0.5879482593609775


,humor,text,pred
0,True,i used to be a banker but i lost interest,0.880655
1,True,i m glad i know sign language it s pretty handy,0.982881
2,True,i don t trust these stairs because they re alw...,0.843077
3,True,did you hear about the guy who got hit in the ...,0.979717
4,True,yesterday i accidentally swallowed some food c...,0.969249
